In [1]:
import re
import warnings
from typing import List
 
import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)
 
warnings.filterwarnings("ignore", category=UserWarning)

### base model

In [2]:
# load the model
BASE_MODEL_NAME = "tiiuae/falcon-7b-instruct"
 
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME, device_map="auto"
)
model = model.eval()
 
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
# model genration config
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    generation_config=generation_config,
)
 
llm = HuggingFacePipeline(pipeline=generation_pipeline)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
print(llm("What is KBase?"))

The current implementation of Falcon calls `torch.scaled_dot_product_attention` directly, this will be deprecated in the future in favor of the `BetterTransformer` API. Please install the latest optimum library with `pip install -U optimum` and call `model.to_bettertransformer()` to benefit from `torch.scaled_dot_product_attention` and future performance optimizations.



KBase is a free, open-source knowledge base software that allows users to create and maintain their own knowledge bases. It is designed to be easy to use and highly customizable, making it a great tool for businesses and individuals alike.


In [4]:
print(llm("How to use KBase narrative?"))


To use KBase narrative, you need to first create a narrative object and then add it to a KViz scene. You can do this by using the `KViz::add_narrative()` function. Once added, you can customize the narrative by setting its properties, such as its text, color, and font. You can also add images and videos to the narrative by using the `KViz::add_image()` and `KViz::add_video()` functions. Finally, you can display the narrative in the scene by calling the `KViz::show()` function.


### fine tuned model

In [6]:
# load the model
TUNED_MODEL_NAME = "fine_tuned_model_kbase_all_data"
 
tuned_model = AutoModelForCausalLM.from_pretrained(
    TUNED_MODEL_NAME, device_map="auto"
)
tuned_model = tuned_model.eval()
 
tuned_model_tokenizer = AutoTokenizer.from_pretrained(TUNED_MODEL_NAME)

tuned_model_generation_pipeline = pipeline(
    model=tuned_model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    generation_config=generation_config,
)
 
tuned_llm = HuggingFacePipeline(pipeline=tuned_model_generation_pipeline)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
print(tuned_llm("What is KBase?"))


KBase is a platform for analyzing and visualizing genomic data. It provides tools and resources to help researchers analyze their own or public genomes, including annotation, comparative analysis, functional enrichment analysis, and visualization of the results. KBase also offers an integrated development environment (IDE) called JupyterLab that allows users to perform these analyses in real-time using interactive code cells.


In [8]:
print(tuned_llm("How to use KBase narrative?"))


To use KBase narrative, you can follow these steps:

1. Open the Narrative interface in your web browser and log in with your KBase account.
2. Click on the 'New Story' button to create a new story.
3. Give your story a title and select a template or start from scratch if desired.
4. Add content to your story by dragging and dropping elements such as text, images, videos, and more into the canvas.
5. Use Markdown syntax to format text and add hyperlinks.
6. Save your changes and publish your story when ready.
7. Share your story with others using various methods such as sharing links, embedding it on websites, or posting it on social media platforms.

KBase provides detailed documentation and resources for users to learn how to effectively use the Narrative platform. You can access them through the help center and community forums.
